In [1]:
#pip install emoji

In [2]:
#pip install natasha

In [3]:
#pip install pymorphy2

In [4]:
#pip install nltk

In [5]:
import pandas as pd
import numpy as np
import re
import emoji

import pymorphy2
from nltk.stem.snowball import SnowballStemmer
from nltk.corpus import stopwords

In [6]:
data = pd.read_csv('tatarstan_message_правка_2.csv', sep = ';', encoding= 'utf-8')
print(data.columns)

data = data[['Категория, присвоенная модератором', 'Описание заявки', 'Последний исполнитель']]
data.columns = ['categories', 'description', 'worker']
data.head(3)
data.loc[18685]['description']

Index(['Дата подачи заявки', 'Долгота', 'Широта', 'Адрес',
       'Категория, присвоенная модератором', 'Описание заявки',
       'Последний исполнитель', 'id исполнителя', 'Статус заявки'],
      dtype='object')


'Требую сделать!'

In [7]:
categories = []
for cat in data.categories:
    if cat not in categories:
        categories.append(cat)
print(categories)
print(len(categories))

['Благоустройство территории', 'Содержание и ремонт муниципальных дорог', 'Поликлиники и больницы', 'Бездомные животные', 'Парки и скверы', 'Мобильная связь', 'Организация дорожного движения', 'Незаконные азартные игры', 'Санитарное состояние', 'Нарушение в наружной рекламе', 'Работа спортивных объектов', 'Капитальный ремонт', 'Общественный транспорт', 'Воздух', 'Садоводческие, огороднические и дачные некоммерческие объединения', 'Детские сады', 'Доступная среда', 'Жилищное строительство', 'Почта', 'Нарушение правил торговли', 'Пенсионное обеспечение', 'Вода', nan, 'Содержание и ремонт федеральных и республиканских дорог', 'Меры социальной поддержки, предоставляемые органами социальной защиты', 'Cвалки', 'Работа учреждений культуры', 'Пособия безработным', 'Социальное обслуживание', 'Объекты культурного наследия', 'Сельское хозяйство', 'Цифровое телевидение', 'Тех.средства и услуги по реабилитации инвалидов за счет средств федерального бюджета', 'Опека, попечительство', 'Ошибки в назва

In [8]:
def preprocess_text(text):
    def replace_three_or_more(text): #remove dublicate of symbol
        pattern = re.compile(r"(.)\1{2,}", re.DOTALL)
        return pattern.sub(r"\1\1", text)

    def emo_to_text(text):  
        emoticon = {'улыбка шутка' : ['\:\)', '\:\-\)', '\:\=\)'],
                    'грусть разочарование' : ['\:\(', '\:\-\(', '\:\=\('],
                    'открытая радость' : ['\=\)', '\=\-\)'],
                    'усмешка хихиканье' : ['\:>', '\:\->', '\:\=>'],
                    'улыбка' : ['\:\]', '\:\-\]', '\:\=\]'],
                    'смех' : ['\:D', '\:\-D', '\:\=D'],
                    'сильный смех' : ['\:DD', '\:\-DD', '\:\=DD'],
                    'сарказм' : ['\:\}', '\:\-\}', '\:\=\}'],
                    'подмигивание заигрывание' : ['\;\)', '\;\-\)', '\;\=\)'],
                    'молчание' : ['\:Х', '\:\-Х', '\:\=Х'],
                    'удивленное разочарование' : ['8\(', '8\-\(', '8\=\('],
                    'изумление' : ['B\-о', 'B\=о'],
                    'радостное удивление' : ['\%\)'],
                    'зевота' : ['\|\-o', '\|\=o'],
                    'смущение' : ['\:S', '\:-S', '\:\=S'],
                    'равнодушие скука' : ['\:\|', '\:\-\|', '\:\=\|'],
                    'недоверие сомнение' : ['\:\/', '\:\-\/', '\:\=\/'],
                    'дразнящее показывание языка' : ['\:ь', '\:\-ь', '\:\=ь'],
                    'поцелуй' : ['\:\*', '\:\^\*', '\:\-Ф'],
                    'шалость дурачество' : ['\:0\)'],
                    'кривая ухмылка' : ['\:7', '\:\-7']}

        for key, value in emoticon.items():
            text = re.sub(r"|".join(value), ' ' + key, text)
        return text

    try:
        text = emo_to_text(text)
        text = emoji.demojize(text, delimiters=("", "")) #преобразует эмодзи-символы в слова
        text = text.lower().replace("ё", "е")
        text = re.sub('((www\.[^\s]+)|(https?://[^\s]+))', 'URL', text)
        text = re.sub('@[^\s]+', 'USER', text)
        text = re.sub('[^a-zA-Zа-яА-Я]+', ' ', text) #удаляем цифры и спецсимволы
        text = replace_three_or_more(text) #удаляет дублирование букв, если число букв более 3
        text = re.sub(' +', ' ', text)
        return text.strip()
    except:
        #print(f'Слово {text} не прошло препроцессинг')
        return '' 

In [9]:
data['description_prep'] = data['description'].apply(preprocess_text)

In [10]:
data['num_words'] = data['description_prep'].apply(str.split, args = (' '))
data['num_words'] = data['num_words'].apply(len)
print(data['num_words'])

0         28
1         36
2         21
3         21
4         39
        ... 
29131    536
29132      1
29133      1
29134      1
29135      1
Name: num_words, Length: 29136, dtype: int64


In [11]:
data = data.loc[data['num_words'] > 3] #удалить строку, если меньше 4-х слов
data.reset_index(inplace=True)
print(data['num_words'])

0         28
1         36
2         21
3         21
4         39
        ... 
23693     22
23694     17
23695     17
23696     41
23697    536
Name: num_words, Length: 23698, dtype: int64


In [12]:
cat_num = dict()
for cat in categories:
    num = np.sum(data['categories'] == cat)
    cat_num.update({cat : num})
print(cat_num)

{'Благоустройство территории': 10953, 'Содержание и ремонт муниципальных дорог': 6232, 'Поликлиники и больницы': 521, 'Бездомные животные': 684, 'Парки и скверы': 385, 'Мобильная связь': 193, 'Организация дорожного движения': 1684, 'Незаконные азартные игры': 10, 'Санитарное состояние': 90, 'Нарушение в наружной рекламе': 1017, 'Работа спортивных объектов': 26, 'Капитальный ремонт': 127, 'Общественный транспорт': 880, 'Воздух': 139, 'Садоводческие, огороднические и дачные некоммерческие объединения': 13, 'Детские сады': 127, 'Доступная среда': 76, 'Жилищное строительство': 34, 'Почта': 43, 'Нарушение правил торговли': 4, 'Пенсионное обеспечение': 2, 'Вода': 26, nan: 0, 'Содержание и ремонт федеральных и республиканских дорог': 39, 'Меры социальной поддержки, предоставляемые органами социальной защиты': 8, 'Cвалки': 304, 'Работа учреждений культуры': 16, 'Пособия безработным': 2, 'Социальное обслуживание': 3, 'Объекты культурного наследия': 15, 'Сельское хозяйство': 18, 'Цифровое телеви

In [13]:
#Фильтрация категорий, в которых менее 10 обращений
for key, value in cat_num.items():
    if value < 10:
        data = data.loc[data['categories'] != key]
data.reset_index(inplace=True)
print(data['num_words'])

0         28
1         36
2         21
3         21
4         39
        ... 
23665     22
23666     17
23667     17
23668     41
23669    536
Name: num_words, Length: 23670, dtype: int64


In [14]:
categories = []
for cat in data.categories:
    if cat not in categories:
        categories.append(cat)
print(categories)
print(len(categories))
""" выводятся из файла модели BERT
cat_path = "flask/categories.txt"
with open(cat_path, 'w') as f:
    for n, cat in enumerate(categories):
        f.write(str(n) + ';' + cat + '\n')"""

['Благоустройство территории', 'Содержание и ремонт муниципальных дорог', 'Поликлиники и больницы', 'Бездомные животные', 'Парки и скверы', 'Мобильная связь', 'Организация дорожного движения', 'Незаконные азартные игры', 'Санитарное состояние', 'Нарушение в наружной рекламе', 'Работа спортивных объектов', 'Капитальный ремонт', 'Общественный транспорт', 'Воздух', 'Садоводческие, огороднические и дачные некоммерческие объединения', 'Детские сады', 'Доступная среда', 'Жилищное строительство', 'Почта', 'Вода', 'Содержание и ремонт федеральных и республиканских дорог', 'Cвалки', 'Работа учреждений культуры', 'Объекты культурного наследия', 'Сельское хозяйство', 'Цифровое телевидение']
26


' выводятся из файла модели BERT\ncat_path = "flask/categories.txt"\nwith open(cat_path, \'w\') as f:\n    for n, cat in enumerate(categories):\n        f.write(str(n) + \';\' + cat + \'\n\')'

In [15]:
#извлечение именованных сущностей
from natasha import (
    Segmenter,
    MorphVocab,
    
    NewsEmbedding,
    NewsMorphTagger,
    NewsSyntaxParser,
    NewsNERTagger,
    
    PER,
    LOC,
    NamesExtractor,

    Doc
)

segmenter = Segmenter()
morph_vocab = MorphVocab()

emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)
syntax_parser = NewsSyntaxParser(emb)
ner_tagger = NewsNERTagger(emb)

names_extractor = NamesExtractor(morph_vocab)

def delete_NER(words):
    nf_words = ' '.join(words)
    per_words = []
    loc_words = []
    doc = Doc(nf_words)

    doc.segment(segmenter)

    doc.tag_ner(ner_tagger)

    for span in doc.spans:
        span.normalize(morph_vocab)
    for span in doc.spans:
        if span.type == 'PER':
            span.extract_fact(names_extractor)
            per_words.append(span.text)
        if span.type == 'LOC':
            span.extract_fact(names_extractor)
            loc_words.append(span.text)
       
    for word in per_words:
        if word in nf_words:
            nf_words = nf_words.replace(word, ' PER ')
    for word in loc_words:
        if word in nf_words:
            nf_words = nf_words.replace(word, ' LOC ')
    words = nf_words.split(' ')
    return words  

In [16]:
#удаление именованных сущностей из наименования исполнителей
worker = []
stops = set(stopwords.words("english")) | set(stopwords.words("russian"))
for row in data['worker']:
    text = str(row)
    text = text.split(' ')
    text = delete_NER(text)
    text = ' '.join(text)
    text = text.replace("ё", "е")
    text = re.sub('[^a-zA-Zа-яА-Я]+', ' ', text) #удаляем цифры и спецсимволы
    text = re.sub(' +', ' ', text).strip()
    words = text.split(' ')
    words = [w.strip() for w in words if (not w in ['PER', 'LOC']) and (len(w) > 1) and w != None and w != '']
    w_ = ''
    for n, w in enumerate(words):
        if w == w_:
            words[n] = ''
        w_ = w
    words = ' '.join(words) #ttext_stem
    if words[-3:] == ' по':
        words = words[:-3]
    if words[-2:] == ' и':
        words = words[:-2]
    if words != None and words.strip() != '' and words.strip() != 'nan':
        worker.append(words)
    else:
        worker.append('Неизвестный исполнитель')
data['worker'] = pd.Series(worker)

In [17]:
wo = []
for w in data.worker:
    if w not in wo:
        wo.append(w)
wo = pd.Series(wo)
print(wo)
print(len(wo))
executor_path = "flask/executors.txt"
with open(executor_path, 'w') as f:
    for n, e in enumerate(wo):
        f.write(str(n) + ';' + e + '\n')

0                                Исполнительный комитет
1                                         Администрация
2                          Министерство здравоохранения
3                       Муниципальное образование город
4      Управление административно технической инспекции
                            ...                        
77            Приикское территориальное управление МЭПР
78    Министерство по делам гражданской обороны чрез...
79             Министерство экологии природных ресурсов
80             Северное территориальное управление МЭПР
81    Палата земельных имущественных отношений Альме...
Length: 82, dtype: object
82


In [18]:
#for n, row in enumerate(data['worker']):
#    if 'заявка решена' in str(row):
#        print(n)

In [19]:
#data.loc[14589]

In [20]:
#data['worker'][14589]

In [21]:
#стемминг
morph = pymorphy2.MorphAnalyzer() #по умолчанию русский язык
stemmer = SnowballStemmer("english")
stops = set(stopwords.words("english")) | set(stopwords.words("russian"))

ttext_stem = list()
for row in data['description_prep']:
    words = row.split(' ')
    words = delete_NER(words) #удаление именованных сущностей: ФИО и гео-данных
    nf_words = list()
    for word in words:
        try:
            #word = word.encode('utf8mb4')
            ru_word = re.sub('[^а-яА-Я]+', ' ', word)
            en_word = re.sub('[^a-zA-Z]+', ' ', word)
            if ru_word == word and word != '': # кириллица
                parse_word = morph.parse(word)[0]
                nf_word = parse_word.normal_form
                nf_words.append(nf_word.strip())
            elif en_word == word and word != '': # латиница
                nf_word = stemmer.stem(word)
                nf_words.append(nf_word.strip())
            else:
                pass #смешанные слова из кириллицы и латиницы не анализируются
        except Exception as e:
                print(f'Слово не преобразовано {e}') 
                continue
    nf_words = [w for w in nf_words if not w in stops]
    
    nf_words = ' '.join(nf_words) #ttext_stem
    nf_words = nf_words.lower().replace("ё", "е")
    ttext_stem.append(nf_words)

In [22]:
if ttext_stem != None:
    ttext_stem = pd.Series(ttext_stem) 
    data['description_stem'] = ttext_stem  
    #data['description_stem'] = '""' + data['description_stem'] + '""'
    print(data['description_stem'])

0        трасса нижнекамск чистополь лесополоса пгт кам...
1        двор дом ленинградский отсутствовать парковочн...
2        добрый день осуществляться уборка снег центр г...
3        добрый день осуществляться вывоз мусор террито...
4        дать участок проезжий часть ул халитовый дом у...
                               ...                        
23665    устранить прицеп магазин цифровик парковка тц ...
23666    производиться уборка детский площадка дом корп...
23667    свалка отход мусор гаражный кооператив автомоб...
23668    здравствовать каждый год пора начать привозить...
23669    сентябрь программа развитие садоводческий движ...
Name: description_stem, Length: 23670, dtype: object


In [23]:
#data = data[['description_stem','categories']]
data.to_csv("tatarstan_message_2.csv", sep = ',',index = False)

In [24]:
#print(data)